In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import datetime as datetime
import requests
import time

load_dotenv()

LASTFM_KEY = os.getenv('LASTFM_KEY')
LASTFM_SECRET = os.getenv('LASTFM_SECRET')
LASTFM_USER = os.getenv('LASTFM_USER')

def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': LASTFM_USER}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = LASTFM_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

# r = lastfm_get({'method': 'chart.gettopartists'})
# r.status_code

def jsonPrint(obj):
    # create a formatted string of the Python JSON object
    result = json.dumps(obj, sort_keys=True, indent=4)
    print(result)

# jsonPrint(r.json())


def lookupTrack(artist, track, album, dict):
    payload = {
        'method': 'track.getInfo',
        'track':  track,
        'artist':  artist
    }
    response = lastfm_get(payload)

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    
    lookupResults = response.json()
    # jsonPrint(lookupResults)

    if 'track' not in lookupResults:
        dict['artist_mbid'].append('N/A')
        dict['artist_name'].append(artist)
        dict['track_name'].append(track)
        dict['album_name'].append(album)
        dict['listeners'].append(0)
        dict['playcount'].append(0)
    else:
        if 'mbid' not in lookupResults['track']['artist']:
            dict['artist_mbid'].append('N/A')
        else:
            dict['artist_mbid'].append(lookupResults['track']['artist']['mbid'])

        dict['artist_name'].append(lookupResults['track']['artist']['name'])
        dict['track_name'].append(lookupResults['track']['name'])
        dict['album_name'].append(album)
        dict['listeners'].append(lookupResults['track']['listeners'])
        dict['playcount'].append(lookupResults['track']['playcount'])



def createTracksDict(dataFrame):
    trackDict = {'artist_mbid': [], 'artist_name': [], 'track_name': [], 'album_name': [], 'listeners': [], 'playcount': []}

    for idx, item in dataFrame.iterrows():
        # print(item['artist_name'] + ' - ' + item['track_name'])
        lookupTrack(item['artist_name'], item['track_name'], item['album_name'], trackDict)
        time.sleep(0.25)

    return trackDict

def writeFile(trackDict, fileName):
    with open(fileName, 'w') as outfile:
        json.dump(trackDict, outfile)

HHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData_20221226.xlsx',sheet_name='HHPlaylist')
CHHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData_20221226.xlsx',sheet_name='CHHPlaylist')

writeFile(createTracksDict(HHPlaylistDF), 'HHTracksDict.json')
time.sleep(1)
writeFile(createTracksDict(CHHPlaylistDF), 'CHHTracksDict.json')

# dict = {}
# lookupTrack('Lil Durk','Mad Max',dict)
# dict = createTracksDict(HHPlaylistDF)
# dict

In [ ]:
# Get Artist API Data
def lookupArtist(artist, dict):
    response = lastfm_get({
        'method': 'artist.getInfo',
        'artist':  artist
    })

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    
    lookupResults = response.json()
    # jsonPrint(lookupResults)

    if 'artist' not in lookupResults:
        dict['artist_mbid'].append('N/A')
        dict['artist_name'].append(artist)
        dict['listeners'].append(0)
        dict['playcount'].append(0)
    else:
        if 'mbid' not in lookupResults['artist']:
            dict['artist_mbid'].append('N/A')
        else:
            dict['artist_mbid'].append(lookupResults['artist']['mbid'])

        dict['artist_name'].append(lookupResults['artist']['name'])
        dict['listeners'].append(lookupResults['artist']['stats']['listeners'])
        dict['playcount'].append(lookupResults['artist']['stats']['playcount'])



def createArtistDict(dataFrame):
    artistDict = {'artist_mbid': [], 'artist_name': [], 'listeners': [], 'playcount': []}

    for idx, item in dataFrame.iterrows():
        # print(item['artist_name'] + ' - ' + item['track_name'])
        lookupArtist(item['artist_name'], artistDict)
        time.sleep(0.25)

    return artistDict


HHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData_20221226.xlsx',sheet_name='HHPlaylist')
CHHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData_20221226.xlsx',sheet_name='CHHPlaylist')

writeFile(createArtistDict(HHPlaylistDF), 'HHArtistDict.json')
time.sleep(1)
writeFile(createArtistDict(CHHPlaylistDF), 'CHHArtistDict.json')

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import datetime as datetime
import requests
import time

load_dotenv()

LASTFM_KEY = os.getenv('LASTFM_KEY')
LASTFM_SECRET = os.getenv('LASTFM_SECRET')
LASTFM_USER = os.getenv('LASTFM_USER')

def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': LASTFM_USER}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = LASTFM_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response
    
    # Get Album API Data
def lookupAlbum(artist, album, dict):
    response = lastfm_get({
        'method': 'album.getInfo',
        'artist':  artist,
        'album':  album
    })

    # if there's an error, just return nothing
    if response.status_code != 200:
        return None

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    
    
    lookupResults = response.json()
    # jsonPrint(lookupResults)

    if 'album' not in lookupResults:
        dict['album_mbid'].append('N/A')
        dict['album_name'].append(album)
        dict['listeners'].append(0)
        dict['playcount'].append(0)
    else:
        if 'mbid' not in lookupResults['album']:
            dict['album_mbid'].append('N/A')
        elif lookupResults['album']['mbid'] == '':
            dict['album_mbid'].append('N/A')
        else:
            dict['album_mbid'].append(lookupResults['album']['mbid'])

        dict['album_name'].append(lookupResults['album']['name'])
        dict['listeners'].append(lookupResults['album']['listeners'])
        dict['playcount'].append(lookupResults['album']['playcount'])

def createAlbumDict(dataFrame):
    albumDict = {'album_mbid': [], 'album_name': [], 'listeners': [], 'playcount': []}

    for idx, item in dataFrame.iterrows():
        # print(item['artist_name'] + ' - ' + item['track_name'])
        lookupAlbum(item['artist_name'], item['album_name'], albumDict)
        time.sleep(0.25)

    return albumDict

def writeFile(trackDict, fileName):
    with open(fileName, 'w') as outfile:
        json.dump(trackDict, outfile)

HHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData_20221226.xlsx',sheet_name='HHPlaylist')
CHHPlaylistDF = pd.read_excel('../Spotify/SpotifyPlaylistData_20221226.xlsx',sheet_name='CHHPlaylist')

writeFile(createAlbumDict(HHPlaylistDF), 'HHAlbumDict.json')
time.sleep(1)
writeFile(createAlbumDict(CHHPlaylistDF), 'CHHAlbumDict.json')

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
import datetime as datetime
import requests
import time

def openFileToDataFrame(fileName):
    with open(fileName, 'r') as openfile:
        file = json.load(openfile)

    df = pd.DataFrame(file)
    return df

getDataDate = datetime.datetime.today().strftime('%Y%m%d')
xlxWriter = pd.ExcelWriter('LastFMData_' + getDataDate + '.xlsx', engine='xlsxwriter')

openFileToDataFrame('HHTracksDict.json').to_excel(xlxWriter, sheet_name='HHTrackData')
openFileToDataFrame('CHHTracksDict.json').to_excel(xlxWriter, sheet_name='CHHTrackData')
openFileToDataFrame('HHArtistDict.json').to_excel(xlxWriter, sheet_name='HHArtistData')
openFileToDataFrame('CHHArtistDict.json').to_excel(xlxWriter, sheet_name='CHHArtistData')
openFileToDataFrame('HHAlbumDict.json').to_excel(xlxWriter, sheet_name='HHAlbumData')
openFileToDataFrame('CHHAlbumDict.json').to_excel(xlxWriter, sheet_name='CHHAlbumData')

xlxWriter.close()